# 모델 평가
 - 지금까지는 주어진 데이터를 분류하는 작업을 진행했습니다. 그리고 그 데이터들은 간단하고 깔끔하여 분류하기 편했습니다. 하지만 세상에 깔끔한 데이터만 있지는 않습니다. 일반 사회에는 데이터가 엉켜 노이즈가 심한 데이터들이 많습니다. 따라서 데이터들의 패턴을 찾아내서 모델을 생성해야 합니다. 신경망에서 학습이란 데이터를 분류하는 일뿐만 아니라 데이터가 지닌 진짜 패턴을 예측하는 일을 진행해야 합니다. 즉, 패턴을 예측해야 합니다.
  - 이러한 데이터들은 100% 완벽하게 분류하기란 쉽지 않습니다. 때문에 모델을 만들었을 떄, 이 모델이 과연 얼마나 타당한지에 대한 평가가 필요합니다. 이 평가에 자주 사용되는 지표로는 정답률, 적합률, 재현률이 있습니다.

# 혼합행렬
 - 실제값과 테스트 데이터를 예측을 통해 얻은 값을 2X2행렬로 나타낸 것을 혼합행렬이라 합니다.
 - (t, y) = (1,1)을 진양성(true positive)(TP), (t, y) = (0,1)을 위양성(false positive)(FP)
 - (t, y) = (1,0)을 위음성(false negative)(FN), (t, y) = (0,0)을 진음성(true negative)(TN) 이라 합니다.
 - 정답률 : (TP + TN) / (TP + FN + FP + TN)
 - 적합률 : TP / (TP + FP)
 - 재현률 : TP / (TP + FN)
 - 일반적으로는 정답률만 구해서 평가합니다.

# 구현 예시(toy problem)
 - sklearn의 dataset을 이용하여 예측 모델을 만들어 보겠습니다. 2개의 클래스를 각각 150개의 데이터를 만들겠습니다.

In [11]:
from sklearn import datasets # 실험용 데이터 셋
from sklearn.utils import shuffle
import tensorflow as tf

N = 300
X, y = datasets.make_moons(N, noise=0.3) # noise란 데이터가 겹치는 깔끔하지 않은 부분을 뜻함.
# 이러한 실험적인 문제를 장난감 문제(toy problem)이라고 함.

In [12]:
from sklearn.model_selection import train_test_split

Y = y.reshape(N, 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8) # 모든 데이터를 훈련용 데이터와 테스트 데이터로 8:2 분류함.

In [13]:
num_hidden = 3 # 은닉층의 차원을 3로 설정

x = tf.placeholder(tf.float32, shape=[None, 2])
t = tf.placeholder(tf.float32, shape=[None, 1])

# 입력층~은닉층
W = tf.Variable(tf.truncated_normal([2, num_hidden]))
b = tf.Variable(tf.zeros([num_hidden]))
h = tf.nn.sigmoid(tf.matmul(x, W) + b)

# 은닉층~출력층
V = tf.Variable(tf.truncated_normal([num_hidden, 1]))
c = tf.Variable(tf.zeros([1]))
y = tf.nn.sigmoid(tf.matmul(h, V) + c)

cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1-t)*tf.log(y-1))
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

In [14]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # 정확히 예측됐는지 확인하기 위해 accuracy를 정의
# correct_prediction의 데이터 타입은 bool형이므로 tf.cast를 통해 부동소수점으로 변환해서 정답 비율을 구할 수 있습니다.

In [15]:
batch_size = 20
n_batches = N

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(500):
    X_, Y_ = shuffle(X_train, Y_train) # 무조건 train데이터만 학습을 진행해야 합니다. test데이터도 학습해버리면 테스트의 의미가 없습니다.
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        
        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end]
        })

In [16]:
accuracy_rate = accuracy.eval(session=sess, feed_dict={
    x: X_test,
    t: Y_test
})
print('accuracy: ', accuracy_rate)

accuracy:  0.81666666


# 케라스로 구현

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [14]:
N = 300
X, y = datasets.make_moons(N, noise=0.3)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.8)

In [17]:
model = Sequential()
model.add(Dense(3, input_dim=2))
model.add(Activation('sigmoid'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.05), metrics=['accuracy']) # 케라스에서는 정확도를 metrics=['accuracy']사용

In [18]:
model.fit(X_train, Y_train, epochs=500, batch_size=20)




Epoch 1/500
240/240 [==============================] - 0s 775us/step - loss: 0.6184 - accuracy: 0.7250
Epoch 2/500
240/240 [==============================] - 0s 54us/step - loss: 0.6139 - accuracy: 0.7208
Epoch 3/500
240/240 [==============================] - 0s 58us/step - loss: 0.6096 - accuracy: 0.7208
Epoch 4/500
240/240 [==============================] - 0s 63us/step - loss: 0.6055 - accuracy: 0.7375
Epoch 5/500
240/240 [==============================] - 0s 79us/step - loss: 0.6010 - accuracy: 0.7417
Epoch 6/500
240/240 [==============================] - 0s 58us/step - loss: 0.5967 - accuracy: 0.7333
Epoch 7/500
240/240 [==============================] - 0s 83us/step - loss: 0.5926 - accuracy: 0.7458
Epoch 8/500
240/240 [==============================] - 0s 54us/step - loss: 0.5883 - accuracy: 0.7458
Epoch 9/500
240/240 [==============================] - 0s 63us/step - loss: 0.5837 - accuracy: 0.7500
Epoch 10/500
240/240 [==============================] - 0s 67us/step - loss: 0

240/240 [==============================] - 0s 46us/step - loss: 0.3670 - accuracy: 0.8458
Epoch 154/500
240/240 [==============================] - 0s 54us/step - loss: 0.3668 - accuracy: 0.8458
Epoch 155/500
240/240 [==============================] - 0s 46us/step - loss: 0.3663 - accuracy: 0.8458
Epoch 156/500
240/240 [==============================] - 0s 46us/step - loss: 0.3664 - accuracy: 0.8458
Epoch 157/500
240/240 [==============================] - 0s 38us/step - loss: 0.3660 - accuracy: 0.8500
Epoch 158/500
240/240 [==============================] - 0s 46us/step - loss: 0.3656 - accuracy: 0.8458
Epoch 159/500
240/240 [==============================] - 0s 50us/step - loss: 0.3654 - accuracy: 0.8458
Epoch 160/500
240/240 [==============================] - 0s 58us/step - loss: 0.3652 - accuracy: 0.8500
Epoch 161/500
240/240 [==============================] - 0s 50us/step - loss: 0.3650 - accuracy: 0.8500
Epoch 162/500
240/240 [==============================] - 0s 46us/step - loss: 

240/240 [==============================] - 0s 63us/step - loss: 0.3550 - accuracy: 0.8542
Epoch 310/500
240/240 [==============================] - 0s 46us/step - loss: 0.3552 - accuracy: 0.8542
Epoch 311/500
240/240 [==============================] - 0s 50us/step - loss: 0.3550 - accuracy: 0.8542
Epoch 312/500
240/240 [==============================] - 0s 50us/step - loss: 0.3552 - accuracy: 0.8542
Epoch 313/500
240/240 [==============================] - 0s 50us/step - loss: 0.3551 - accuracy: 0.8542
Epoch 314/500
240/240 [==============================] - 0s 50us/step - loss: 0.3551 - accuracy: 0.8542
Epoch 315/500
240/240 [==============================] - 0s 58us/step - loss: 0.3552 - accuracy: 0.8542
Epoch 316/500
240/240 [==============================] - 0s 42us/step - loss: 0.3552 - accuracy: 0.8583
Epoch 317/500
240/240 [==============================] - 0s 46us/step - loss: 0.3551 - accuracy: 0.8542
Epoch 318/500
240/240 [==============================] - 0s 46us/step - loss: 

240/240 [==============================] - 0s 50us/step - loss: 0.3531 - accuracy: 0.8542
Epoch 466/500
240/240 [==============================] - 0s 46us/step - loss: 0.3529 - accuracy: 0.8542
Epoch 467/500
240/240 [==============================] - 0s 50us/step - loss: 0.3530 - accuracy: 0.8542
Epoch 468/500
240/240 [==============================] - 0s 54us/step - loss: 0.3529 - accuracy: 0.8542
Epoch 469/500
240/240 [==============================] - 0s 54us/step - loss: 0.3532 - accuracy: 0.8542
Epoch 470/500
240/240 [==============================] - 0s 50us/step - loss: 0.3530 - accuracy: 0.8542
Epoch 471/500
240/240 [==============================] - 0s 58us/step - loss: 0.3527 - accuracy: 0.8542
Epoch 472/500
240/240 [==============================] - 0s 67us/step - loss: 0.3528 - accuracy: 0.8542
Epoch 473/500
240/240 [==============================] - 0s 54us/step - loss: 0.3529 - accuracy: 0.8542
Epoch 474/500
240/240 [==============================] - 0s 54us/step - loss: 

In [20]:
loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

60/60 [==============================] - 0s 617us/step
[0.2200174242258072, 0.9666666388511658]


# 딥러닝
 - 세상에는 더욱 복잡한 데이터들이 존재합니다. 이러한 데이터들의 패턴을 파악하기 위해서는 두 가지 방법이 있습니다.
 - 1) 은닉층의 뉴런 수를 늘린다. 2) 은닉층의 수를 늘린다.
 - 사실 은닉층이 1개만 있어도 은닉층에 해당하는 뉴런의 수가 충분하다면 어떠한 함수도 근사할 수 있다는 시벤코의 정리가 있습니다.
 - 하지만, 은닉층의 뉴런의 수를 늘리는 일보다 은닉층 자체의 수를 늘리는 것이 파라미터 효율에 더 좋습니다.
 - 은닉층의 개수를 늘리는 일은 신경망의 층을 더 깊게 만드는데, 이렇게 깊은 층을 이루는 네트워크를 심층 신경망(Deep neural networks)라 합니다. 그리고 심층 신경망을 학습시키는 기법을 딥러닝 또는 심층학습이라고 합니다.

# 구현 예시
 - MNIST라는 데이터 셋을 사용하여 구현을 진행해보겠습니다.
 - MNIST는 신경망 모델의 예측 정확도를 비교하는 데 자주 사용되는 베치마크 테스트용 데이터 셋이며, 0에 9까지의 손글씨 숫자가 그려진 7만장의 이미지 데이터(학습용 6만장, 테스트용 1만장)으로 구성돼 있습니다.

In [18]:
from sklearn import datasets
mnist = datasets.fetch_mldata('MNIST original', data_home='.')
# data_home에 지정한 디렉터리에 MNIST의 합축 파일 다운로드(.은 현재 스크립트를 실행한 현재 디렉토리)
# 데이터는 mnist.data(각 이미지를 그레이 스케일 형태의 값으로 수치화한 데이터), mnist.target(실제 숫자값이 들어 있는 데이터)으로 나뉨.

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


In [20]:
import numpy as np

n = len(mnist.data)
N = 10000 # MNIST의 부분적인 데이터로 실험
indices = np.random.permutation(range(n))[:N] # 무작위로 N장 선정
X = mnist.data[indices]
y = mnist.target[indices]
Y = np.eye(10)[y.astype(int)] # 1-of-k 표현으로 변환

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

n_in = len(X[0]) # 784
n_hidden = 200
n_out = len(Y[0]) # 10

model = Sequential()
model.add(Dense(n_hidden, input_dim=n_in))
model.add(Activation('sigmoid'))

model.add(Dense(n_out))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])

epochs = 1000
batch_size = 100

model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size)

Using TensorFlow backend.





Epoch 1/1000
8000/8000 [==============================] - 1s 107us/step - loss: 1.7059 - accuracy: 0.4899
Epoch 2/1000
8000/8000 [==============================] - 0s 58us/step - loss: 1.0128 - accuracy: 0.7660
Epoch 3/1000
8000/8000 [==============================] - 1s 90us/step - loss: 0.7660 - accuracy: 0.8296
Epoch 4/1000
8000/8000 [==============================] - 0s 58us/step - loss: 0.6358 - accuracy: 0.8577
Epoch 5/1000
8000/8000 [==============================] - 1s 98us/step - loss: 0.5575 - accuracy: 0.8734
Epoch 6/1000
8000/8000 [==============================] - 0s 54us/step - loss: 0.4968 - accuracy: 0.8890
Epoch 7/1000
8000/8000 [==============================] - 2s 223us/step - loss: 0.4543 - accuracy: 0.8974
Epoch 8/1000
8000/8000 [==============================] - 3s 385us/step - loss: 0.4221 - accuracy: 0.9068
Epoch 9/1000
8000/8000 [==============================] - 3s 379us/step - loss: 0.3939 - accuracy: 0.9134
Epoch 10/1000
8000/8000 [=======================

8000/8000 [==============================] - 0s 54us/step - loss: 0.0984 - accuracy: 0.9790
Epoch 73/1000
8000/8000 [==============================] - 0s 57us/step - loss: 0.0978 - accuracy: 0.9791
Epoch 74/1000
8000/8000 [==============================] - 0s 59us/step - loss: 0.0972 - accuracy: 0.9796
Epoch 75/1000
8000/8000 [==============================] - 1s 66us/step - loss: 0.0966 - accuracy: 0.9794
Epoch 76/1000
8000/8000 [==============================] - 0s 52us/step - loss: 0.0961 - accuracy: 0.9797
Epoch 77/1000
8000/8000 [==============================] - 0s 54us/step - loss: 0.0955 - accuracy: 0.9797
Epoch 78/1000
8000/8000 [==============================] - 0s 53us/step - loss: 0.0952 - accuracy: 0.9800
Epoch 79/1000
8000/8000 [==============================] - 1s 163us/step - loss: 0.0948 - accuracy: 0.9797
Epoch 80/1000
8000/8000 [==============================] - 2s 284us/step - loss: 0.0941 - accuracy: 0.9801
Epoch 81/1000
8000/8000 [==============================] -

8000/8000 [==============================] - 0s 59us/step - loss: 0.0705 - accuracy: 0.9855
Epoch 149/1000
8000/8000 [==============================] - 0s 55us/step - loss: 0.0703 - accuracy: 0.9855
Epoch 150/1000
8000/8000 [==============================] - 0s 45us/step - loss: 0.0701 - accuracy: 0.9858
Epoch 151/1000
8000/8000 [==============================] - 0s 52us/step - loss: 0.0698 - accuracy: 0.9851
Epoch 152/1000
8000/8000 [==============================] - 0s 58us/step - loss: 0.0695 - accuracy: 0.9859
Epoch 153/1000
8000/8000 [==============================] - 0s 49us/step - loss: 0.0693 - accuracy: 0.9856
Epoch 154/1000
8000/8000 [==============================] - 0s 48us/step - loss: 0.0690 - accuracy: 0.9854
Epoch 155/1000
8000/8000 [==============================] - 0s 47us/step - loss: 0.0688 - accuracy: 0.9860
Epoch 156/1000
8000/8000 [==============================] - 0s 55us/step - loss: 0.0686 - accuracy: 0.9858
Epoch 157/1000
8000/8000 [==========================

8000/8000 [==============================] - 0s 51us/step - loss: 0.0566 - accuracy: 0.9884
Epoch 225/1000
8000/8000 [==============================] - 0s 53us/step - loss: 0.0565 - accuracy: 0.9885
Epoch 226/1000
8000/8000 [==============================] - 0s 53us/step - loss: 0.0563 - accuracy: 0.9884
Epoch 227/1000
8000/8000 [==============================] - 0s 50us/step - loss: 0.0562 - accuracy: 0.9885
Epoch 228/1000
8000/8000 [==============================] - 0s 51us/step - loss: 0.0561 - accuracy: 0.9885
Epoch 229/1000
8000/8000 [==============================] - 0s 50us/step - loss: 0.0560 - accuracy: 0.9886
Epoch 230/1000
8000/8000 [==============================] - 0s 59us/step - loss: 0.0558 - accuracy: 0.9887
Epoch 231/1000
8000/8000 [==============================] - 0s 49us/step - loss: 0.0556 - accuracy: 0.9887
Epoch 232/1000
8000/8000 [==============================] - 0s 47us/step - loss: 0.0555 - accuracy: 0.9887
Epoch 233/1000
8000/8000 [==========================

...

8000/8000 [==============================] - 0s 44us/step - loss: 0.0241 - accuracy: 0.9960
Epoch 757/1000
8000/8000 [==============================] - 0s 53us/step - loss: 0.0240 - accuracy: 0.9962
Epoch 758/1000
8000/8000 [==============================] - 0s 43us/step - loss: 0.0240 - accuracy: 0.9962
Epoch 759/1000
8000/8000 [==============================] - 0s 47us/step - loss: 0.0240 - accuracy: 0.9962
Epoch 760/1000
8000/8000 [==============================] - 0s 46us/step - loss: 0.0239 - accuracy: 0.9962
Epoch 761/1000
8000/8000 [==============================] - 0s 50us/step - loss: 0.0239 - accuracy: 0.9962
Epoch 762/1000
8000/8000 [==============================] - 0s 50us/step - loss: 0.0239 - accuracy: 0.9964
Epoch 763/1000
8000/8000 [==============================] - 0s 51us/step - loss: 0.0239 - accuracy: 0.9962
Epoch 764/1000
8000/8000 [==============================] - 0s 55us/step - loss: 0.0238 - accuracy: 0.9965
Epoch 765/1000
8000/8000 [==========================

8000/8000 [==============================] - 1s 91us/step - loss: 0.0221 - accuracy: 0.9966
Epoch 833/1000
8000/8000 [==============================] - 2s 228us/step - loss: 0.0221 - accuracy: 0.9966
Epoch 834/1000
8000/8000 [==============================] - 3s 354us/step - loss: 0.0221 - accuracy: 0.9966
Epoch 835/1000
8000/8000 [==============================] - 2s 264us/step - loss: 0.0221 - accuracy: 0.9967
Epoch 836/1000
8000/8000 [==============================] - 2s 304us/step - loss: 0.0220 - accuracy: 0.9966
Epoch 837/1000
8000/8000 [==============================] - 3s 326us/step - loss: 0.0220 - accuracy: 0.9966
Epoch 838/1000
8000/8000 [==============================] - 2s 298us/step - loss: 0.0219 - accuracy: 0.9966
Epoch 839/1000
8000/8000 [==============================] - 3s 313us/step - loss: 0.0219 - accuracy: 0.9966
Epoch 840/1000
8000/8000 [==============================] - 2s 278us/step - loss: 0.0219 - accuracy: 0.9969
Epoch 841/1000
8000/8000 [==================

8000/8000 [==============================] - 0s 51us/step - loss: 0.0204 - accuracy: 0.9973
Epoch 909/1000
8000/8000 [==============================] - 0s 51us/step - loss: 0.0204 - accuracy: 0.9973
Epoch 910/1000
8000/8000 [==============================] - 0s 56us/step - loss: 0.0203 - accuracy: 0.9973
Epoch 911/1000
8000/8000 [==============================] - 0s 54us/step - loss: 0.0203 - accuracy: 0.9974
Epoch 912/1000
8000/8000 [==============================] - 0s 51us/step - loss: 0.0203 - accuracy: 0.9974
Epoch 913/1000
8000/8000 [==============================] - 1s 67us/step - loss: 0.0203 - accuracy: 0.9974
Epoch 914/1000
8000/8000 [==============================] - 0s 56us/step - loss: 0.0203 - accuracy: 0.9974
Epoch 915/1000
8000/8000 [==============================] - 0s 54us/step - loss: 0.0202 - accuracy: 0.9974
Epoch 916/1000
8000/8000 [==============================] - 0s 49us/step - loss: 0.0202 - accuracy: 0.9974
Epoch 917/1000
8000/8000 [==========================

8000/8000 [==============================] - 0s 57us/step - loss: 0.0190 - accuracy: 0.9976
Epoch 985/1000
8000/8000 [==============================] - 0s 50us/step - loss: 0.0189 - accuracy: 0.9977
Epoch 986/1000
8000/8000 [==============================] - 0s 52us/step - loss: 0.0189 - accuracy: 0.9977
Epoch 987/1000
8000/8000 [==============================] - 1s 64us/step - loss: 0.0189 - accuracy: 0.9979
Epoch 988/1000
8000/8000 [==============================] - 0s 50us/step - loss: 0.0189 - accuracy: 0.9977
Epoch 989/1000
8000/8000 [==============================] - 0s 52us/step - loss: 0.0189 - accuracy: 0.9979
Epoch 990/1000
8000/8000 [==============================] - 0s 50us/step - loss: 0.0189 - accuracy: 0.9977
Epoch 991/1000
8000/8000 [==============================] - 0s 56us/step - loss: 0.0188 - accuracy: 0.9979
Epoch 992/1000
8000/8000 [==============================] - 0s 60us/step - loss: 0.0188 - accuracy: 0.9979
Epoch 993/1000
8000/8000 [==========================

AttributeError: 'Sequential' object has no attribute 'evalutate'

In [23]:
loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

2000/2000 [==============================] - 0s 117us/step
[0.3051393163204193, 0.9135000109672546]


# 결과
 - 지금은 은닉층의 뉴런 개수를 200개로 했을 때, 결과입니다. 뉴런 개수를 늘려서 다시 실험을 진행해보면 정답률이 올라간다 싶더라도 일정 이상 뉴런 수가 되면 더 이상 정답률이 증가하지 않을 뿐더러 오히려 감소할 수 있습니다. 게다가 뉴런의 수가 증가할 수록 계산 속도가 오래 걸린다는 것을 알 수 있습니다. 따라서 정답률을 올리기 위해 무작정 뉴런의 수를 올리는 것은 바람직하지 않다는 의미입니다. 동일한 예측 정확도를 얻어낼 수 있다면 뉴런의 수는 작을수록 좋습니다.
 - 그렇다면 은닉층의 개수를 늘려서 모델을 구현해보겠습니다.

In [26]:
model = Sequential()
model.add(Dense(n_hidden, input_dim=n_in))
model.add(Activation('sigmoid'))

model.add(Dense(n_hidden))
model.add(Activation('sigmoid'))

model.add(Dense(n_hidden))
model.add(Activation('sigmoid'))

model.add(Dense(n_hidden))
model.add(Activation('sigmoid'))

model.add(Dense(n_out))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])

epochs = 1000
batch_size = 100

model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size)

loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

Epoch 1/1000
8000/8000 [==============================] - 1s 117us/step - loss: 2.3257 - accuracy: 0.1037
Epoch 2/1000
8000/8000 [==============================] - 1s 78us/step - loss: 2.3020 - accuracy: 0.1085
Epoch 3/1000
8000/8000 [==============================] - 1s 67us/step - loss: 2.3011 - accuracy: 0.1106
Epoch 4/1000
8000/8000 [==============================] - 1s 75us/step - loss: 2.3001 - accuracy: 0.1126
Epoch 5/1000
8000/8000 [==============================] - 1s 80us/step - loss: 2.2994 - accuracy: 0.1171
Epoch 6/1000
8000/8000 [==============================] - 1s 73us/step - loss: 2.2982 - accuracy: 0.1194
Epoch 7/1000
8000/8000 [==============================] - 1s 66us/step - loss: 2.2978 - accuracy: 0.1249
Epoch 8/1000
8000/8000 [==============================] - 1s 75us/step - loss: 2.2969 - accuracy: 0.1305
Epoch 9/1000
8000/8000 [==============================] - 1s 78us/step - loss: 2.2959 - accuracy: 0.1293
Epoch 10/1000
8000/8000 [=============================

8000/8000 [==============================] - 1s 79us/step - loss: 1.4523 - accuracy: 0.5621
Epoch 79/1000
8000/8000 [==============================] - 1s 79us/step - loss: 1.4195 - accuracy: 0.5801
Epoch 80/1000
8000/8000 [==============================] - 1s 76us/step - loss: 1.3880 - accuracy: 0.5874
Epoch 81/1000
8000/8000 [==============================] - 1s 77us/step - loss: 1.3580 - accuracy: 0.5956
Epoch 82/1000
8000/8000 [==============================] - 1s 83us/step - loss: 1.3300 - accuracy: 0.6086
Epoch 83/1000
8000/8000 [==============================] - 1s 82us/step - loss: 1.3037 - accuracy: 0.6130
Epoch 84/1000
8000/8000 [==============================] - 1s 72us/step - loss: 1.2781 - accuracy: 0.6209
Epoch 85/1000
8000/8000 [==============================] - 1s 79us/step - loss: 1.2538 - accuracy: 0.6225
Epoch 86/1000
8000/8000 [==============================] - 1s 71us/step - loss: 1.2312 - accuracy: 0.6456
Epoch 87/1000
8000/8000 [==============================] - 1

8000/8000 [==============================] - 1s 78us/step - loss: 0.4772 - accuracy: 0.9013
Epoch 155/1000
8000/8000 [==============================] - 1s 75us/step - loss: 0.4702 - accuracy: 0.9020
Epoch 156/1000
8000/8000 [==============================] - 1s 73us/step - loss: 0.4649 - accuracy: 0.9025
Epoch 157/1000
8000/8000 [==============================] - 1s 88us/step - loss: 0.4599 - accuracy: 0.9038
Epoch 158/1000
8000/8000 [==============================] - 1s 71us/step - loss: 0.4540 - accuracy: 0.9070
Epoch 159/1000
8000/8000 [==============================] - 1s 73us/step - loss: 0.4497 - accuracy: 0.9087
Epoch 160/1000
8000/8000 [==============================] - 1s 80us/step - loss: 0.4416 - accuracy: 0.9094
Epoch 161/1000
8000/8000 [==============================] - 1s 70us/step - loss: 0.4377 - accuracy: 0.9119
Epoch 162/1000
8000/8000 [==============================] - 1s 87us/step - loss: 0.4311 - accuracy: 0.9125
Epoch 163/1000
8000/8000 [==========================

8000/8000 [==============================] - 1s 76us/step - loss: 0.2031 - accuracy: 0.9616
Epoch 231/1000
8000/8000 [==============================] - 1s 66us/step - loss: 0.2033 - accuracy: 0.9611
Epoch 232/1000
8000/8000 [==============================] - 1s 65us/step - loss: 0.2015 - accuracy: 0.9614
Epoch 233/1000
8000/8000 [==============================] - 1s 74us/step - loss: 0.1986 - accuracy: 0.9615
Epoch 234/1000
8000/8000 [==============================] - 1s 66us/step - loss: 0.1969 - accuracy: 0.9620
Epoch 235/1000
8000/8000 [==============================] - 1s 67us/step - loss: 0.1967 - accuracy: 0.9628
Epoch 236/1000
8000/8000 [==============================] - 1s 65us/step - loss: 0.1950 - accuracy: 0.9626
Epoch 237/1000
8000/8000 [==============================] - 1s 69us/step - loss: 0.1928 - accuracy: 0.9630
Epoch 238/1000
8000/8000 [==============================] - 1s 79us/step - loss: 0.1915 - accuracy: 0.9626
Epoch 239/1000
8000/8000 [==========================

8000/8000 [==============================] - 1s 79us/step - loss: 0.1274 - accuracy: 0.9760
Epoch 307/1000
8000/8000 [==============================] - 1s 69us/step - loss: 0.1260 - accuracy: 0.9760
Epoch 308/1000
8000/8000 [==============================] - 1s 81us/step - loss: 0.1252 - accuracy: 0.9758
Epoch 309/1000
8000/8000 [==============================] - 1s 73us/step - loss: 0.1250 - accuracy: 0.9760
Epoch 310/1000
8000/8000 [==============================] - 1s 83us/step - loss: 0.1239 - accuracy: 0.9761
Epoch 311/1000
8000/8000 [==============================] - 1s 71us/step - loss: 0.1230 - accuracy: 0.9765
Epoch 312/1000
8000/8000 [==============================] - 1s 81us/step - loss: 0.1221 - accuracy: 0.9764
Epoch 313/1000
8000/8000 [==============================] - 1s 71us/step - loss: 0.1213 - accuracy: 0.9761
Epoch 314/1000
8000/8000 [==============================] - 1s 80us/step - loss: 0.1208 - accuracy: 0.9769
Epoch 315/1000
8000/8000 [==========================

...

8000/8000 [==============================] - 1s 75us/step - loss: 0.0257 - accuracy: 0.9965
Epoch 687/1000
8000/8000 [==============================] - 1s 80us/step - loss: 0.0255 - accuracy: 0.9964
Epoch 688/1000
8000/8000 [==============================] - 1s 79us/step - loss: 0.0254 - accuracy: 0.9962
Epoch 689/1000
8000/8000 [==============================] - 1s 79us/step - loss: 0.0254 - accuracy: 0.9965
Epoch 690/1000
8000/8000 [==============================] - 1s 78us/step - loss: 0.0253 - accuracy: 0.9964
Epoch 691/1000
8000/8000 [==============================] - 1s 77us/step - loss: 0.0252 - accuracy: 0.9965
Epoch 692/1000
8000/8000 [==============================] - 1s 70us/step - loss: 0.0252 - accuracy: 0.9965
Epoch 693/1000
8000/8000 [==============================] - 1s 75us/step - loss: 0.0251 - accuracy: 0.9965
Epoch 694/1000
8000/8000 [==============================] - 1s 64us/step - loss: 0.0250 - accuracy: 0.9964
Epoch 695/1000
8000/8000 [==========================

8000/8000 [==============================] - 1s 77us/step - loss: 0.0206 - accuracy: 0.9973
Epoch 763/1000
8000/8000 [==============================] - 1s 73us/step - loss: 0.0205 - accuracy: 0.9973
Epoch 764/1000
8000/8000 [==============================] - 1s 66us/step - loss: 0.0205 - accuracy: 0.9973
Epoch 765/1000
8000/8000 [==============================] - 1s 65us/step - loss: 0.0204 - accuracy: 0.9973
Epoch 766/1000
8000/8000 [==============================] - 1s 67us/step - loss: 0.0204 - accuracy: 0.9973
Epoch 767/1000
8000/8000 [==============================] - 1s 67us/step - loss: 0.0203 - accuracy: 0.9973
Epoch 768/1000
8000/8000 [==============================] - 1s 63us/step - loss: 0.0203 - accuracy: 0.9973
Epoch 769/1000
8000/8000 [==============================] - 1s 101us/step - loss: 0.0202 - accuracy: 0.99730s - loss: 0.0
Epoch 770/1000
8000/8000 [==============================] - 1s 90us/step - loss: 0.0202 - accuracy: 0.9973
Epoch 771/1000
8000/8000 [===========

8000/8000 [==============================] - 1s 69us/step - loss: 0.0172 - accuracy: 0.9976
Epoch 839/1000
8000/8000 [==============================] - 1s 75us/step - loss: 0.0172 - accuracy: 0.9976
Epoch 840/1000
8000/8000 [==============================] - 1s 69us/step - loss: 0.0172 - accuracy: 0.9976
Epoch 841/1000
8000/8000 [==============================] - 1s 76us/step - loss: 0.0171 - accuracy: 0.9976
Epoch 842/1000
8000/8000 [==============================] - 1s 70us/step - loss: 0.0171 - accuracy: 0.9976
Epoch 843/1000
8000/8000 [==============================] - 1s 70us/step - loss: 0.0171 - accuracy: 0.9976
Epoch 844/1000
8000/8000 [==============================] - 1s 82us/step - loss: 0.0170 - accuracy: 0.9976
Epoch 845/1000
8000/8000 [==============================] - 1s 74us/step - loss: 0.0170 - accuracy: 0.9976
Epoch 846/1000
8000/8000 [==============================] - 1s 123us/step - loss: 0.0170 - accuracy: 0.9976
Epoch 847/1000
8000/8000 [=========================

8000/8000 [==============================] - 1s 75us/step - loss: 0.0147 - accuracy: 0.9979
Epoch 915/1000
8000/8000 [==============================] - 1s 72us/step - loss: 0.0147 - accuracy: 0.9980
Epoch 916/1000
8000/8000 [==============================] - 1s 87us/step - loss: 0.0147 - accuracy: 0.9980
Epoch 917/1000
8000/8000 [==============================] - 1s 89us/step - loss: 0.0146 - accuracy: 0.9979
Epoch 918/1000
8000/8000 [==============================] - 1s 70us/step - loss: 0.0146 - accuracy: 0.9980
Epoch 919/1000
8000/8000 [==============================] - 1s 76us/step - loss: 0.0146 - accuracy: 0.9980
Epoch 920/1000
8000/8000 [==============================] - 1s 73us/step - loss: 0.0146 - accuracy: 0.9980
Epoch 921/1000
8000/8000 [==============================] - 1s 70us/step - loss: 0.0145 - accuracy: 0.9980
Epoch 922/1000
8000/8000 [==============================] - 1s 120us/step - loss: 0.0145 - accuracy: 0.9980
Epoch 923/1000
8000/8000 [=========================

8000/8000 [==============================] - 1s 69us/step - loss: 0.0128 - accuracy: 0.9983
Epoch 991/1000
8000/8000 [==============================] - 1s 81us/step - loss: 0.0128 - accuracy: 0.9983
Epoch 992/1000
8000/8000 [==============================] - 1s 72us/step - loss: 0.0128 - accuracy: 0.9983
Epoch 993/1000
8000/8000 [==============================] - 1s 74us/step - loss: 0.0128 - accuracy: 0.9983
Epoch 994/1000
8000/8000 [==============================] - 1s 76us/step - loss: 0.0127 - accuracy: 0.9983
Epoch 995/1000
8000/8000 [==============================] - 1s 81us/step - loss: 0.0127 - accuracy: 0.9983
Epoch 996/1000
8000/8000 [==============================] - 1s 76us/step - loss: 0.0127 - accuracy: 0.9983
Epoch 997/1000
8000/8000 [==============================] - 1s 88us/step - loss: 0.0127 - accuracy: 0.9983
Epoch 998/1000
8000/8000 [==============================] - 1s 124us/step - loss: 0.0127 - accuracy: 0.9984
Epoch 999/1000
8000/8000 [=========================

# 결과
 - 결과를 보시면 알겠지만 층이 많을수록 복잡한 패턴이 잘 표현될 줄 알았는데 딱히 그렇지도 않다는 사실을 알 수 있습니다. 오히려 은닉층이 4개인 위의 결과를 보시면 전보다 학습되지 않았다는 사실을 알 수 있습니다.
 - 위의 실험을 통해 은닉층의 뉴런 수나 은닉층 자체의 개수를 무작정 늘린다고 데이터의 패턴을 더욱 잘 분석할 수 있는 것은 아니라는 시실을 알 수 있었습니다. 하지만 모델이 깊어지면 깊어질수록 분명히 표현하고 분류할 수 있는 패턴은 많아지는 것은 사실입니다. 무슨 문제 때문에 학습이 잘 이뤄지지 않았고 어떻게 하면 해결할 수 있는지에 대해 생각해야 합니다.

# 학습시킬 때 발생하는 문제점
 - 1) 경사 소실 문제
 - 모델을 학습시킬 때 최적해를 구하기 위해 각 매개변수의 경사를 구해야 합니다. 이 때 경사 소실이란 경사가 0이되어 버려 경사가 없어지는 현상을 뜻합니다. 경사가 소실되어 버리면 오차역전파법을 제대로 실행할 수 없습니다.
 - 이는 활성화 함수 중 시그모이드 함수일 때 자주 발생하는 문제인데, 시그모이드 함수의 도함수는 x=0일 때 최댓값이 0.25입니다. 즉, 은닉층이 N개일 때 1보다 작은 수를 N번 곱해줘 오차항이 점점 0에 가까워진다는 문제가 발생합니다.
 - 따라서 미분해도 값이 작아지지 않는 활성함수를 찾아야 합니다.
 - 2) 오버피팅 문제(과학습, 과잉적합)
 - 데이터를 지나치게 학습한 상태가 되는 것을 의미합니다. 주어진 데이터에만 너무 집중해 모델이 학습되면 데이터의 실제 분포와는 다른 패턴 분류의 형태가 될 가능성이 높아집니다. 오차함수 E값이 최소가 되도록 매개변수 값을 변경해 학습했지만 마냥 E값을 최소화하며 오버피팅될 가능성이 있어 무조건 최소화하면 안 된다는 것을 의미합니다. 이와 반대의 경우로 언더피팅(underfitting)이 있습니다.